# Answer Generation Using BERT



In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from keras import backend
import torch
import torch.nn as nn
import re
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import BertTokenizer
import random
import math

/home/kavitha/anaconda3/envs/vqa-xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-20 15:51:19.787709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
vocab_size = 4914
max_len = 1000
heads = 12
epochs = 999999
lr = 0.0001
clip = True
share = 'all'
norm = 'pre'
dim = 312
drop = 0.0
n_layers = 4
batch_size = 40
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device_name)
device = torch.device(device_name)
save_dir = './'

cuda:0


## Data Path

In [ ]:
Train_CSV = './Dataset/Train/QA_Train.csv'
Train_Images = './Dataset/Train/Train_Images/'
Validation_CSV = './Dataset/Valid/QA_Valid.csv'
Validation_Images = './Dataset/Valid/Valid_Images/'
Test_CSV = './Dataset/Test/QA_Test.csv'
Test_Images = './Dataset/Test/Test_Images/'

## Image Feature

In [ ]:
Model_Path = './Image-Feature-Extraction-Model-960.h5'

In [ ]:
Image_Feature_Extraction_Model = tf.keras.models.load_model(Model_Path)
feature_func = backend.function([Image_Feature_Extraction_Model.input],[Image_Feature_Extraction_Model.layers[-2].output])

2023-03-20 15:51:21.227429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 15:51:21.228990: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
def imagePreprocess(img):
  #resize
  resized_img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

  return resized_img

In [ ]:
def getImageFeature(img_dir,img_name):
  img_path = img_dir+img_name+'.jpg'
  img = cv2.imread(img_path)
  pre_processed_img = imagePreprocess(img)
  img_feature = feature_func(np.array([pre_processed_img]))[0][0]
  return [math.ceil(x)+100 for x in img_feature]

## Question Feature

In [ ]:
def textPreprocess(txt):
  #lower case
  txt = txt.lower()

  txt = re.sub('-',' ',txt)

  return  re.sub('[^a-zA-Z0-9]',' ',txt)

In [ ]:
Vocab_Path = './bert-tokenizer'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(Vocab_Path)

In [ ]:
def getTokenizedText(txt):
  return list(tokenizer(txt)['input_ids'][1:-1])

## Answer Generation

In [ ]:
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [ ]:
class Embeddings(nn.Module):
    def __init__(self,max_len):
        super(Embeddings, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, 128, padding_idx=0)
        self.word_embeddings_2 = nn.Linear(128, dim, bias=False)
        self.position_embeddings = nn.Embedding(max_len, dim)
        self.type_embeddings = nn.Embedding(3, dim)
        self.LayerNorm = nn.LayerNorm(dim, eps=1e-12)
        self.dropout = nn.Dropout(drop)
        self.len = max_len
    def forward(self, input_ids, segment_ids, position_ids=None):
        #print(input_ids)
        input_ids = torch.cuda.LongTensor(input_ids)
        segment_ids = torch.cuda.LongTensor(segment_ids)
        if position_ids is None:
            position_ids = torch.arange(self.len, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        words_embeddings = self.word_embeddings(input_ids)
        words_embeddings = self.word_embeddings_2(words_embeddings)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.type_embeddings(segment_ids)
        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [ ]:
class MultiHeadedSelfAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.proj_q = nn.Linear(dim, dim)
        self.proj_k = nn.Linear(dim, dim)
        self.proj_v = nn.Linear(dim, dim)
        self.drop = nn.Dropout(drop)
        self.scores = None
        self.n_heads = heads
    def forward(self, x, mask):
        q, k, v = self.proj_q(x), self.proj_k(x), self.proj_v(x)
        q, k, v = (self.split_last(x, (self.n_heads, -1)).transpose(1, 2) for x in [q, k, v])
        scores = q @ k.transpose(-2, -1) / np.sqrt(k.size(-1))
        if mask is not None:
            mask = mask[:, None, None, :].float()
            scores -= 10000.0 * (1.0 - mask)
        scores = self.drop(F.softmax(scores, dim=-1))
        h = (scores @ v).transpose(1, 2).contiguous()
        h = self.merge_last(h, 2)
        self.scores = scores
        return h
    def split_last(self, x, shape):
        shape = list(shape)
        assert shape.count(-1) <= 1
        if -1 in shape:
            shape[shape.index(-1)] = int(x.size(-1) / -np.prod(shape))
        return x.view(*x.size()[:-1], *shape)
    def merge_last(self, x, n_dims):
        s = x.size()
        assert n_dims > 1 and n_dims < len(s)
        return x.view(*s[:-n_dims], -1)

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim*4)
        self.fc2 = nn.Linear(dim*4, dim)
    def forward(self, x):
        return self.fc2(gelu(self.fc1(x)))

In [ ]:
class BertLayer(nn.Module):
    def __init__(self, share='all', norm='pre'):
        super(BertLayer, self).__init__()
        self.share = share
        self.norm_pos = norm
        self.norm1 = nn.LayerNorm(dim, eps=1e-12)
        self.norm2 = nn.LayerNorm(dim, eps=1e-12)
        self.drop1 = nn.Dropout(drop)
        self.drop2 = nn.Dropout(drop)
        if self.share == 'ffn':
            self.attention = nn.ModuleList([MultiHeadedSelfAttention() for _ in range(n_layers)])
            self.proj = nn.ModuleList([nn.Linear(dim, dim) for _ in range(n_layers)])
            self.feedforward = PositionWiseFeedForward()
        elif self.share == 'att':
            self.attention = MultiHeadedSelfAttention()
            self.proj = nn.Linear(dim, dim)
            self.feedforward = nn.ModuleList([PositionWiseFeedForward() for _ in range(n_layers)])
        elif self.share == 'all':
            self.attention = MultiHeadedSelfAttention()
            self.proj = nn.Linear(dim, dim)
            self.feedforward = PositionWiseFeedForward()
        elif self.share == 'none':
            self.attention = nn.ModuleList([MultiHeadedSelfAttention() for _ in range(n_layers)])
            self.proj = nn.ModuleList([nn.Linear(dim, dim) for _ in range(n_layers)])
            self.feedforward = nn.ModuleList([PositionWiseFeedForward() for _ in range(n_layers)])
    def forward(self, hidden_states, attention_mask, layer_num):
        attention_mask = torch.cuda.LongTensor(attention_mask)
        if self.norm_pos == 'pre':
            if isinstance(self.attention, nn.ModuleList):
                h = self.proj[layer_num](self.attention[layer_num](self.norm1(hidden_states), attention_mask))
            else:
                h = self.proj(self.attention(self.norm1(hidden_states), attention_mask))
            out = hidden_states + self.drop1(h)
            if isinstance(self.feedforward, nn.ModuleList):
                h = self.feedforward[layer_num](self.norm1(out))
            else:
                h = self.feedforward(self.norm1(out))
            out = out + self.drop2(h)
        if self.norm_pos == 'post':
            if isinstance(self.attention, nn.ModuleList):
                h = self.proj[layer_num](self.attention[layer_num](hidden_states, attention_mask))
            else:
                h = self.proj(self.attention(hidden_states, attention_mask))
            out = self.norm1(hidden_states + self.drop1(h))
            if isinstance(self.feedforward, nn.ModuleList):
                h = self.feedforward[layer_num](out)
            else:
                h = self.feedforward(out)
            out = self.norm2(out + self.drop2(h))
        return out

In [ ]:
class Transformer(nn.Module):
    def __init__(self,max_len=max_len):
        super().__init__()
        self.embed = Embeddings(max_len)
        self.blocks = BertLayer(share='none', norm='pre')
        self.n_layers = n_layers
    def forward(self, x, seg, mask):
        h = self.embed(x, seg)
        for i in range(self.n_layers):
            h = self.blocks(h, mask, i)
        return h

In [ ]:
class VQAModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.transformer = Transformer(max_len)
        self.fc2 = nn.Linear(dim, dim)
        self.norm = nn.LayerNorm(dim, eps=1e-12)
        embed_weight = self.transformer.embed.word_embeddings.weight
        n_vocab, embed_dim = embed_weight.size()
        self.decoder = nn.Linear(dim, embed_dim, bias=False)
        self.decoder_2 = nn.Linear(embed_dim, n_vocab, bias=False)
        self.decoder_2.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))
    def forward(self, input_ids, segment_ids, input_mask, masked_pos):
        h = self.transformer(input_ids, segment_ids, input_mask)
        masked_pos = torch.cuda.LongTensor(masked_pos)[:, :, None].expand(-1, -1, h.size(-1))
        h_masked = torch.gather(h, 1, masked_pos)
        h_masked = self.norm(gelu(self.fc2(h_masked)))
        logits_lm = self.decoder_2(self.decoder(h_masked)) + self.decoder_bias
        return logits_lm

In [ ]:
def extract_data(file,flag=0):
    imag, ques, answ = [],[],[]
    df = pd.read_csv(file,header=None)
    #print(df.head)
    for line in df.index:
        imag.append(df[0][line])
        ques.append(textPreprocess(df[1][line]))
        if(flag==0):
            answ.append(textPreprocess(df[2][line]))
    return imag, ques, answ

In [ ]:
def gene_data(text_file,flag=0):
    _imag, _ques, _answ = extract_data(text_file,flag)
    ge_imag, ge_ques, ge_answ = [], [], []
    for i in range(len(_imag)):
        ge_imag.append(_imag[i])
        ge_ques.append(_ques[i])
        if(flag==0):
            ge_answ.append(_answ[i])
    state = np.random.get_state()
    np.random.shuffle(ge_imag)
    np.random.set_state(state)
    np.random.shuffle(ge_ques)
    if(flag==0):
        np.random.set_state(state)
        np.random.shuffle(ge_answ)
        return ge_imag, ge_ques, ge_answ
    else:
        return ge_imag,ge_ques

In [ ]:
train_imag, train_ques, train_answ = gene_data(Train_CSV)
valid_imag, valid_ques, valid_answ = gene_data(Validation_CSV)
test_imag, test_ques = gene_data(Test_CSV,1)

In [ ]:
CLS = getTokenizedText('[CLS]')
SEP = getTokenizedText('[SEP]')
MASK = getTokenizedText('[MASK]')
CLS,SEP,MASK

([2], [3], [4])

In [ ]:
'''def data_loader(image_path,imag, ques, answ, batch_size, gen_len):
    count = 0
    while count < len(imag):
        batch = []
        if batch_size < len(imag) - count:
            size = batch_size
        else: 
            size = len(imag) - count
        for _ in range(size):
            imag_name = imag[count]
            part1 = getImageFeature(image_path,imag_name)
            part2 = getTokenizedText(ques[count])
            part3 = getTokenizedText(answ[count])
            tokens = []
            tokens = CLS + part1 + SEP + part2 + SEP + part3 + SEP
            masked_tokens, masked_pos = [], []
            num = random.randint(len(part1+part2)+3,len(part1+part2+part3)+3)
            masked_pos.append(num)
            masked_tokens.append(tokens[num])
            masked_weights = [1]*len(masked_tokens)
            tokens[num] = MASK[0]
            for i in range(num+1,len(tokens)):
                tokens[i] = 0
            segment_ids = [0]*(len(part1)+2) + [1]*(len(part2)+1) + [2]*(num-len(part1+part2)-2)
            input_mask = [1]*(num+1)
            n_pad = gen_len - num - 1
            #print("Before-->",tokens)
            tokens.extend([0]*(gen_len - len(tokens)))
            segment_ids.extend([0]*n_pad)
            input_mask.extend([0]*n_pad)
            #print("After-->",tokens)
            batch.append((tokens, segment_ids, input_mask, masked_tokens, masked_pos, masked_weights))
            count += 1
        yield batch'''
def data_loader(image_path,imag, ques, answ, batch_size, gen_len):
    count = 0
    batch = []
    temp_batch = []
    while count < len(imag):
        if batch_size < len(imag) - count:
            size = batch_size
        else: 
            size = len(imag) - count
        size = size - len(batch)
        x =0 
        while (x<size):
            imag_name = imag[count]
            part1 = getImageFeature(image_path,imag_name)
            part2 = getTokenizedText(ques[count])
            part3 = getTokenizedText(answ[count])
            for i in range(len(part3)+1):
              tokens = []
              tokens = CLS + part1 + SEP + part2 + SEP + part3 + SEP
              masked_tokens, masked_pos = [], []
              num = len(part1+part2)+3+i
              masked_pos.append(num)
              masked_tokens.append(tokens[num])
              masked_weights = [1]*len(masked_tokens)
              tokens[num] = MASK[0]
              for i in range(num+1,len(tokens)):
                tokens[i] = 0
              segment_ids = [0]*(len(part1)+2) + [1]*(len(part2)+1) + [2]*(num-len(part1+part2)-2)
              input_mask = [1]*(num+1)
              n_pad = gen_len - num - 1
              #print("Before-->",tokens)
              tokens.extend([0]*(gen_len - len(tokens)))
              segment_ids.extend([0]*n_pad)
              input_mask.extend([0]*n_pad)
              #print("After-->",tokens)
              x += 1
              if(x<size):
                batch.append((tokens, segment_ids, input_mask, masked_tokens, masked_pos, masked_weights))
              else:
                temp_batch.append((tokens, segment_ids, input_mask, masked_tokens, masked_pos, masked_weights))
            count += 1
        if(len(batch)==0):
          if(count>=len(imag)):
            return
          continue
        yield batch
        if(len(temp_batch)<batch_size):
          batch = temp_batch.copy()
          temp_batch = []
        else:
          batch = temp_batch[:batch_size].copy()
          temp_batch = temp_batch[batch_size:].copy()

In [ ]:
def findLoss(model, batch):
    tokens, segment_ids, input_mask, masked_tokens, masked_pos, masked_weights = zip(*batch)
    logits_lm = model(tokens, segment_ids, input_mask, masked_pos)
    masked_tokens, masked_weights = torch.cuda.LongTensor(masked_tokens), torch.cuda.LongTensor(masked_weights)
    loss_lm = nn.CrossEntropyLoss(reduction='none')(logits_lm.transpose(1, 2), masked_tokens)
    loss_lm = (loss_lm*masked_weights.float()).mean()
    return loss_lm

In [ ]:
def train(model, iterator, optimizer, epoch, mode='class'):
    model.train()
    epoch_loss, count = 0, 0
    iter_bar = tqdm(iterator, desc='Training')
    for _, batch in enumerate(iter_bar):
        count += 1
        optimizer.zero_grad()
        loss = findLoss(model, batch)
        loss = loss.mean()
        loss.backward()
        if clip:
            nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
        optimizer.step()
        iter_bar.set_description('Iter (loss=%5.3f)'%loss.item())
        epoch_loss += loss.item()
    return epoch_loss / count

In [ ]:
def valid(model, iterator, epoch):
    model.eval()
    epoch_loss, count = 0, 0
    with torch.no_grad():
        iter_bar = tqdm(iterator, desc='Validation')
        for _, batch in enumerate(iter_bar):
            count += 1
            loss = findLoss(model, batch)
            loss = loss.mean()
            iter_bar.set_description('Iter (loss=%5.3f)'%loss.item())
            epoch_loss += loss.item()
    return epoch_loss / count

In [ ]:
def get_answ(model, dict_op, tokens, segment_ids, input_mask):
    model.eval()
    with torch.no_grad():
        pred = model([tokens], [segment_ids], [input_mask])
    out = dict_op[int(np.argsort(pred.cpu())[:,-1:][0][0])]
    return out

In [ ]:
def train_model(model,imag_t,ques_t,answ_t,imag_v,ques_v,answ_v,log_name,threshold,dict_op,loss_mode='gene'):
    log_file = save_dir+log_name+'.txt'
    with open(log_file, 'w') as log_f:
        log_f.write('epoch, train_loss, valid_loss\n')
    optimizer = optim.Adam(model.parameters(),lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=3, verbose=True)
    stop = 0
    loss_list = []
    result_dict = {}
    for epoch in range(epochs):
        train_iterator = data_loader(Train_Images,imag_t,ques_t,answ_t, batch_size, max_len)
        valid_iterator = data_loader(Validation_Images,imag_v,ques_v,answ_v, batch_size, max_len)
        print('Epoch: ' + str(epoch+1))
        train_loss = train(model, train_iterator, optimizer, epoch, loss_mode)
        valid_loss = valid(model, valid_iterator, epoch)
        scheduler.step(valid_loss)
        loss_list.append(valid_loss)
        with open(log_file, 'a') as log_f:
            log_f.write('{epoch},{train_loss: 3.5f},{valid_loss: 3.5f}\n'.format(epoch=epoch+1, train_loss=train_loss, valid_loss=valid_loss))
        if valid_loss == min(loss_list) and valid_loss < threshold:
            stop = 0
        else:
            stop += 1
            if stop > 10:
                break
        torch.save(model.state_dict(),"./Models-960-v2/model-"+str(epoch+1)+".pt")
    return model

In [ ]:
dict_op = {value:key for key, value in dict(tokenizer.vocab).items()}

In [ ]:
len(dict_op)

4914

In [ ]:
for i in range(10):
  print(i,"::",dict_op[i])

0 :: [PAD]
1 :: [UNK]
2 :: [CLS]
3 :: [SEP]
4 :: [MASK]
5 :: "
6 :: &
7 :: '
8 :: (
9 :: )


In [ ]:
#torch.cuda.memory_summary(device=device, abbreviated=False)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
VQA_Model = VQAModel().to(device)
VQA_Model = train_model(VQA_Model,train_imag, train_ques, train_answ, valid_imag, valid_ques, valid_answ,'log-960-v2',100,dict_op,loss_mode='gene')

Epoch: 1


Iter (loss=0.065): : 1042it [21:09,  1.22s/it]
Iter (loss=0.802): : 176it [02:29,  1.18it/s]


Epoch: 2


Iter (loss=0.023): : 1042it [21:18,  1.23s/it]
Iter (loss=0.902): : 176it [02:28,  1.19it/s]


Epoch: 3


...

Iter (loss=0.000): : 1042it [21:07,  1.22s/it]
Iter (loss=1.527): : 176it [02:27,  1.19it/s] 


Epoch: 19


Iter (loss=0.000): : 1042it [21:04,  1.21s/it]
Iter (loss=1.519): : 176it [02:27,  1.19it/s] 


Epoch: 20


Iter (loss=0.000): : 1042it [21:04,  1.21s/it]
Iter (loss=1.511): : 176it [02:27,  1.19it/s] 
